# 해야할 것

`1` BERTopic model을 불러와서 학습

`2` 모델 학습 후 각 기사 데이터의 토픽 추론(`BertTopic`)

`3` 추론한 토픽 컬럼을 생성

`4` 사용자가 언어를 선택하면 그 언어에 맞게끔 데이터가 바뀌는 함수 작성

`5` 기사의 핵심어 산출을 위한 `KeyBERT` model load

`6` 사용자가 해당 기사의 제목을 선택하면 요약문(`KoBART`)과 원문을 함께 보이게 하기 위해서 `데이터 프레임`을 `return` 하는 함수 작성

`7` 그 후 사용자가 토픽, 날짜, 지역을 선택하면 해당 지역의 기사를 위험도(`일단 내가 임의로 설정`)순으로 기사가 보이게끔

`8` 그리고 사용자가 어떤 지역의 언어를 선택하면 그 언어로 핵심어, 요약문, 원문을 번역해주는 함수 작성(`mBart-50`)

# BERTopic

`-`  일단 데이터로 모델 학습 후 로컬에서 모델을 불러와야하므로 `코랩 환경`에서 수행

## 0. import

```python
!pip install bertopic
!pip install bertopic[visualization]
```

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import pandas as pd

## 1. 데이터 로드

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/Colab Notebooks/뉴스빅데이터해커톤/test

/content/drive/MyDrive/Colab Notebooks/뉴스빅데이터해커톤/test


`-` 일단 test case 이므로 100개 데이터만 가지고 수행하자

In [41]:
df = pd.read_excel("raw data.xlsx").sample(n = 100)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 17014 to 7534
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   뉴스 식별자             100 non-null    float64
 1   일자                 100 non-null    int64  
 2   언론사                100 non-null    object 
 3   기고자                98 non-null     object 
 4   제목                 100 non-null    object 
 5   통합 분류1             100 non-null    object 
 6   통합 분류2             90 non-null     object 
 7   통합 분류3             77 non-null     object 
 8   사건/사고 분류1          100 non-null    object 
 9   사건/사고 분류2          29 non-null     object 
 10  사건/사고 분류3          10 non-null     object 
 11  인물                 67 non-null     object 
 12  위치                 94 non-null     object 
 13  기관                 100 non-null    object 
 14  키워드                100 non-null    object 
 15  특성추출(가중치순 상위 50개)  100 non-null    object 
 16  본문                 10

`-` 쓸데없는 컬럼 삭제

In [43]:
d_col = ["언론사", "기고자", "통합 분류1", "통합 분류2", "통합 분류3",
         "사건/사고 분류1", "사건/사고 분류2", "사건/사고 분류3",
         "인물", "위치", "기관", "키워드","특성추출(가중치순 상위 50개)", "분석제외 여부"]

df.drop(d_col, axis = 1, inplace =True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 17014 to 7534
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   뉴스 식별자  100 non-null    float64
 1   일자      100 non-null    int64  
 2   제목      100 non-null    object 
 3   본문      100 non-null    object 
 4   URL     100 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.7+ KB


`-` 일자를 `날짜 데이터 ` 형식으로 변경

In [45]:
df.일자 = [str(i)[0:4] + "-" + str(i)[4:6] + "-" + str(i)[6:] for i in df.일자]
df.일자 = pd.to_datetime(df.일자)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 17014 to 7534
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   뉴스 식별자  100 non-null    float64       
 1   일자      100 non-null    datetime64[ns]
 2   제목      100 non-null    object        
 3   본문      100 non-null    object        
 4   URL     100 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 4.7+ KB


In [47]:
df.head()

,뉴스 식별자,일자,제목,본문,URL
17014,8.100101e+06,2023-09-05,전라북도 “댐 방류 피해 지원 지역 확대 건의”,[KBS 전주]\n전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 ...,https://news.kbs.co.kr/news/view.do?ncd=776628...
7259,8.100401e+06,2023-10-14,격화하는 충돌...언론인 인명 피해 속출,[앵커]\n이스라엘과 하마스와의 충돌이 격화하면서 현장을 취재하는 언론인들의 인명 ...,https://www.ytn.co.kr/_ln/0104_202310142106034159
486,1.100101e+06,2023-11-04,하마스 “이스라엘의 학교 겨냥 공격으로 20명 사망”,팔레스타인 가자지구의 학교를 겨냥한 이스라엘군의 공습으로 20명이 사망했다고 팔레스...,https://www.khan.co.kr/world/world-general/art...
7814,2.100701e+06,2023-10-12,이스라엘-팔레스타인 교전에 사망자 2500명 대다수가 민간인,[헤럴드경제=박자연 기자]이스라엘과 팔레스타인 무장 정파 하마스 교전이 엿새째 격화...,http://biz.heraldcorp.com/view.php?ud=20231012...
6690,1.101101e+06,2023-10-16,이스라엘 북부도 '제2의 전선' 긴장 고조... 국경 마을 28곳에 대피령,이스라엘 중서부에 위치한 가자지구에서 이스라엘군(IDF)이 조만간 지상 작전을 개시...,https://www.hankookilbo.com/News/Read/A2023101...


## 2. 전처리 (생략)

`-` 아직 형태소 분석기 뭐 쓸지 안정했으니 생략하자.

`-` [전처리+모델링 참고링크](https://gangcheol.github.io/mysite2/posts/DX/2023-10-26-02.%20BERTopic.html#mecab-%EC%84%A4%EC%B9%98)

## 3.  모델링

In [49]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                                   nr_topics=10, ## 토픽 개수
                                  top_n_words=10, ## 추출할 토픽 당 상위 단어 수
                                  calculate_probabilities=True) ## Documnent 별로 특정 Topic에 속하는 확류를 계산

In [50]:
topics, probs = model.fit_transform(df["본문"])

## 4. 결과 확인

In [51]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16,-1_앵커_바랍니다_재난대응_eu,"[앵커, 바랍니다, 재난대응, eu, 보다, 모로코, 후쿠시마, 기자, 고위대표는,...",[호세프 보렐 유럽연합(EU) 외교안보 고위대표는 가자지구로 훨씬 더 많은 구호품 ...
1,0,12,0_지진이_규모_알레르기_아침,"[지진이, 규모, 알레르기, 아침, 것이, 지난, 오후, 기자, 생육, 소중히]",[【평창】평창군은 18일 군 청사에서 지진에 대한 경각심을 고취하고 재난 대응능력을...
2,1,39,1_이스라엘_팔레스타인_현지시간_하마스,"[이스라엘, 팔레스타인, 현지시간, 하마스, 가자지구, 이스라엘과, 이스라엘이, 하...",[이츠하크 헤르초그 이스라엘 대통령이 “가자지구에서 민간인 사상자를 줄이는 데 집중...
3,2,18,2_더불어민주당_이재명_단식_대표가,"[더불어민주당, 이재명, 단식, 대표가, 대표의, 국회, 가운데, 이날, 문재인, ...",[이재명 더불어민주당 대표의 단식이 11일째를 지나는 가운데 이 대표의 단식이 언제...
4,3,15,3_원주_고용노동부_흉기난동_있다,"[원주, 고용노동부, 흉기난동, 있다, 이후, 오전, 통해, 위한, 대표의, 따르면]",[2024학년도 대학수학능력시험(수능)이 실시되는 다음 달 16일 관공서와 기업 등...


## 4.  기존 데이터의 토픽 정보 추가

In [70]:
df["topic"] = topics

## 6. 결과 데이터 저장

In [73]:
df.to_excel("rowdata(size=100).xlsx")

***

# KeyBERT 핵심어 추출(`다시 로컬로 넘어옴`)

In [15]:
import pandas as pd

In [17]:
df = pd.read_excel("rowdata(size=100).xlsx")

In [19]:
df.head()

,Unnamed: 0,뉴스 식별자,일자,제목,본문,URL,topic
0,17014,8.100101e+06,2023-09-05,전라북도 “댐 방류 피해 지원 지역 확대 건의”,[KBS 전주]\n전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 ...,https://news.kbs.co.kr/news/view.do?ncd=776628...,0
1,7259,8.100401e+06,2023-10-14,격화하는 충돌...언론인 인명 피해 속출,[앵커]\n이스라엘과 하마스와의 충돌이 격화하면서 현장을 취재하는 언론인들의 인명 ...,https://www.ytn.co.kr/_ln/0104_202310142106034159,1
2,486,1.100101e+06,2023-11-04,하마스 “이스라엘의 학교 겨냥 공격으로 20명 사망”,팔레스타인 가자지구의 학교를 겨냥한 이스라엘군의 공습으로 20명이 사망했다고 팔레스...,https://www.khan.co.kr/world/world-general/art...,1
3,7814,2.100701e+06,2023-10-12,이스라엘-팔레스타인 교전에 사망자 2500명 대다수가 민간인,[헤럴드경제=박자연 기자]이스라엘과 팔레스타인 무장 정파 하마스 교전이 엿새째 격화...,http://biz.heraldcorp.com/view.php?ud=20231012...,1
4,6690,1.101101e+06,2023-10-16,이스라엘 북부도 '제2의 전선' 긴장 고조... 국경 마을 28곳에 대피령,이스라엘 중서부에 위치한 가자지구에서 이스라엘군(IDF)이 조만간 지상 작전을 개시...,https://www.hankookilbo.com/News/Read/A2023101...,1


In [4]:
#pip install keybert

In [31]:
from keybert import KeyBERT
from transformers import BartForConditionalGeneration

`-` pretrained 된 `digit82/kobart-summarization`을 기본 모델로 만들고 Keybert 객체로 만들어준다.

In [32]:
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
kw_model = KeyBERT(model)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


`-` 일단 1개 데이터

In [223]:
text = df["본문"][0]

In [224]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=15)
keywords

[('홍수위로부터', 0.6274),
 ('홍수', 0.5986),
 ('환경분쟁조정을', 0.5284),
 ('피해가', 0.4223),
 ('계획', 0.384),
 ('지역까지', 0.3446),
 ('지원', 0.3175),
 ('지역', 0.3125),
 ('지역에만', 0.3111),
 ('따르면', 0.305),
 ('전라북도가', 0.3046),
 ('남원', 0.2877),
 ('집중호우', 0.2854),
 ('2천20년', 0.2676),
 ('하류', 0.2666)]

`-` 확인했으니 루프로 모든 데이터에 적용한다음에 컬럼 만들어서 붙이자.

In [225]:
keyword_list = []

In [229]:
for i in range(len(df)) :
    text = df["본문"][i]
    temp = [i[0] for i in keywords]
    keyword_list.append(temp)

## 사용자 정의함수 작성

In [231]:
def keyword_extract(df) :
    model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
    kw_model = KeyBERT(model)
    keyword_list = []
    for i in range(len(df)) :
        text = df["본문"][i]
        keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=15)
        temp = [i[0] for i in keywords]
        keyword_list.append(temp)
    df["keyword"] = keyword_list
    df["keyword"] = [" ".join(i) for i in df["keyword"]]
    return df

In [232]:
df = keyword_extract(df)
df.head()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


,Unnamed: 0,뉴스 식별자,일자,제목,본문,URL,topic,keyword,summary_text,위험도,city
49,19388,2.100311e+06,2023-08-29,尹 카눈 피해 심각 지자체 3곳 특별재난지역 추가 선포,윤석열 대통령이 제6호 태풍 카눈으로 인해 대규모 피해가 발생한 강원 고성군 경북 ...,http://www.sedaily.com/NewsView/29TLG4GC12,3,홍수위로부터 홍수 환경분쟁조정을 피해가 계획 지역까지 지원 지역 지역에만 따르면 전...,제6호 태풍 카눈으로 인해 대규모 피해가 발생한 강원 고성군 경북 경주시 산내면 경...,12.492217,경상북도
60,19480,2.100501e+06,2023-08-29,'살인 예고글' 전국서 479건 23명 구속,신림동 흉기난동 이후 229건 검거 \n \n 대구국제공항에 대한 '폭탄테러'와 공...,http://www.fnnews.com/news/202308290954412819,3,언론인들의 기자 이스라엘과 카메라 레바논 기자가 사격 하마스와의 폭발음과 피해가 보...,대구국제공항에 대한 '폭탄테러'와 공항 이용객 불특정 다수를 상대로 '흉기난동'이 ...,1.554176,전라북도
54,19282,2.100851e+06,2023-08-29,10호 태풍 '담레이' 일본 삿포로 해상에서 소멸,제10호 태풍 '담레이' 경로. 담레이는 29일 오후 3시에 소멸했다. [자료=기상...,https://www.ajunews.com/view/20230829171154152,1,팔레스타인 이스라엘군의 20명과 알사프타위 가자지구 하마스 부상자 수십명이 가자지구...,29일 낮 낮 일본 삿포로 인근 해상에서 소멸된 제10호 태풍 '담레이'가 29일 ...,12.258479,대전광역시
44,19317,7.100501e+06,2023-08-29,"한난, '2023년 재난대응 안전한국훈련' 시행",한국지역난방공사는 29일 경기 파주시 지사에서 행정안전부가 주관하는 '2023년 제...,http://www.etnews.com/20230829000284,-1,이스라엘 이스라엘과 팔레스타인 2500명을 사망자가 사망자 하마스 1300명 사망자...,"한국지역난방공사는 29일 경기 파주시 지사에서 중앙부처와 지방자치단체, 공공기관, ...",8.469362,세종특별자치시
9,18922,1.100701e+06,2023-08-30,日 “간토대지진 조선인 학살 사실 파악 기록 없어”,"일본 정부가 올해로 100주년을 맞은 간토대지진 당시 군대와 경찰, 민간인으로 이뤄...",http://www.segye.com/content/html/2023/08/30/2...,2,이스라엘 이스라엘이 이스라엘군 레바논과 가자지구에서 국경지대 인접한 지상 작전을 일...,"일본 정부가 올해로 100주년을 맞은 간토대지진 당시 군대와 경찰, 민간인으로 이뤄...",2.554163,대구광역시


# KoBART 요약문

## import

In [63]:
#!pip install transformers
#conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [64]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [66]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
summary_model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


## test

In [67]:
text = df["본문"][0]

In [69]:
text

'[KBS 전주]\n전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 확대하는 방안을 환경부에 건의했습니다.\n\n현행법에 따르면 계획 홍수위로부터 5킬로미터 안까지만 지원 범위로 정하고 있어 상류 지역에만 혜택이 적용됩니다.\n\n실제 2천20년 집중호우 때 댐 방류로 남원, 임실 등 하류 지역 5곳에 피해가 나 주민들이 환경분쟁조정을 신청했지만,..'

In [70]:
len(text)

200

## 문서 토큰화

In [72]:
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'전라전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 확대하는 방안을 환경부에 건의하였다.'

## 문서 요약 생성

In [73]:
summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
s_text =  tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

In [74]:
s_text

'전라전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 확대하는 방안을 환경부에 건의하였다.'

## 사용자 정의함수 작성

`-` 데이터 100개인데도 시간 좀 많이 걸림

In [78]:
def text_Summarization(df) :
    tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
    summary_model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
    summary_text = []
    for i in range(len(df)) :
        text = df["본문"][i]
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
        s_text =  tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        summary_text.append(s_text)
    df["summary_text"] = summary_text
    return df

In [79]:
df = text_Summarization(df)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


```python
summary_text_ids = model.generate(
    input_ids=torch.tensor([input_ids]),
    length_penalty=2.0, # 길이에 대한 penalty. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
    max_length=128,     # 요약문의 최대 길이 설정
    min_length=32,      # 요약문의 최소 길이 설정
    num_beams=4,        # 문장 생성시 다음 단어를 탐색하는 영역의 개수
    length_panelty = 2.0 # 길이에 대한 panelty, 1보다 작은 경우 더 짧은 문장을 생성
)
```

## 비교

In [82]:
print(df["본문"][0])

[KBS 전주]
전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 확대하는 방안을 환경부에 건의했습니다.

현행법에 따르면 계획 홍수위로부터 5킬로미터 안까지만 지원 범위로 정하고 있어 상류 지역에만 혜택이 적용됩니다.

실제 2천20년 집중호우 때 댐 방류로 남원, 임실 등 하류 지역 5곳에 피해가 나 주민들이 환경분쟁조정을 신청했지만,..


In [84]:
print(df["summary_text"][0])

전라전라북도가 홍수 때 댐 방류로 피해가 나면 하류 지역까지 지원을 확대하는 방안을 환경부에 건의하였다.


***

# mBart-50

`-` 자 이제 피날레

`-` 사용자가 어떤 언어를 선택하면 해당 언어로 바꿔주기

`-` [참고링크](https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt)

## import

In [93]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

## model load

`-` load 하는데 시간 좀 많이 걸림

In [94]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

## 언어dic 작성

In [103]:
lang_dict = {"Arabic" : "ar_AR", "Czech": "cs_CZ", "German" : "de_DE", "English" : "en_XX", 
             "Spanish" :  "es_XX", "Estonian" : "et_EE", "Finnish" : "fi_FI", "French" : 'fr_XX', 
            'Gujarati': 'gu_IN', 'Hindi' : 'hi_IN', 'Italian' : 'it_IT', 'Japanese' : 'ja_XX', 
            'Kazakh' : 'kk_KZ', 'Korean' : 'ko_KR', 'Lithuanian' : 'lt_LT', 'Latvian' : 'lv_LV', 
            'Burmese' : 'my_MM', 'Nepali' : 'ne_NP', 'Dutch' : 'nl_XX', 
            'Romanian' : 'ro_RO', 'Russian' : 'ru_RU', 'Sinhala' : 'si_LK', 
            'Turkish' : 'tr_TR', 'Vietnamese' : 'vi_VN', 
            'Chinese' : 'zh_CN', 'Afrikaans' : 'af_ZA', 
            'Azerbaijani' : 'az_AZ', 'Bengali' : 'bn_IN', 
            'Persian' : 'fa_IR', 'Hebrew' : 'he_IL', 'Croatian' : 'hr_HR', 'Indonesian' : 'id_ID', 
            'Georgian' : 'ka_GE', 'Khmer' : 'km_KH', 'Macedonian' : 'mk_MK', 'Malayalam' : 'ml_IN', 
            'Mongolian' : 'mn_MN', 'Marathi' : 'mr_IN', 'Polish' : 'pl_PL', 
            'Pashto' : 'ps_AF', 'Portuguese' : 'pt_XX', 'Swedish' : 'sv_SE', 'Swahili' : 'sw_KE', 
            'Tamil' : 'ta_IN', 'Telugu' : 'te_IN', 'Thai' : 'th_TH', 'Tagalog' : 'tl_XX', 'Ukrainian' : 'uk_UA', 
            'Urdu' : 'ur_PK', 'Xhosa' : 'xh_ZA', 'Galician' : 'gl_ES', 'Slovene' : 'sl_SI'}

## test

### 기사별 가짜 위험도 생성

In [106]:
import numpy as np

In [111]:
sampl = np.random.uniform(low=0.5, high=13.3, size=100)

In [116]:
df["위험도"] = sampl

In [121]:
df = df.sort_values("일자")

### 기사별 가짜 지역 생성

In [126]:
sido = pd.read_excel("시도단위 코드.xlsx")["시도"].tolist()
sido

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [131]:
import random
city = [random.choice(sido) for _ in range(100)]
df["city"] = city

In [132]:
df.city.value_counts()

city
강원도        10
대전광역시      10
광주광역시      10
경기도         9
제주특별자치도     8
경상남도        8
세종특별자치시     7
경상북도        6
인천광역시       6
충청남도        6
서울특별시       6
대구광역시       3
충청북도        3
울산광역시       2
부산광역시       2
전라북도        2
전라남도        2
Name: count, dtype: int64

### 사용자가 다음과 같이 조건을 걸었다고 가정

In [133]:
df["일자"]

49   2023-08-29
60   2023-08-29
54   2023-08-29
44   2023-08-29
9    2023-08-30
        ...    
19   2023-11-03
77   2023-11-03
68   2023-11-03
2    2023-11-04
45   2023-11-06
Name: 일자, Length: 100, dtype: datetime64[ns]

`-` 강원도, 토픽1, 날짜 : `2023-08-29 ~ 2023-09-29`

In [212]:
input_city = "강원도"
input_topic = 1
input_date_start = "2023-10-14"
input_date_end = "2023-11-01"
select_language = "English"

In [213]:
return_df = df.loc[df["city"] == input_city]
return_df = return_df[return_df["topic"] == input_topic]
return_df = return_df.loc[return_df.일자.between(input_date_start, input_date_end)]
return_df.reset_index(drop = True, inplace = True)

### 번역 테스트

In [155]:
article_kr = return_df["본문"][0]
title_kr = return_df["제목"][0]
topic_kr = return_df["topic"][0]
date = return_df["일자"][0]
language = lang_dict[select_language]

In [169]:
tokenizer.src_lang = "ko_KR" ## 디폴트 한국어 고정

encoded_article_kr = tokenizer(article_kr, return_tensors="pt")
encoded_title_kr = tokenizer(title_kr, return_tensors="pt")
#encoded_topic_kr = tokenizer(article_kr, return_tensors="pt")

generated_article = model.generate(
                        **encoded_article_kr,
                        forced_bos_token_id = tokenizer.lang_code_to_id[language])

generated_title = model.generate(
                        **encoded_title_kr,
                        forced_bos_token_id = tokenizer.lang_code_to_id[language])
#generated_topic = model.generate(
 #                       **encoded_topic_kr,
  #                      forced_bos_token_id = tokenizer.lang_code_to_id[language])

In [202]:
df

,Unnamed: 0,뉴스 식별자,일자,제목,본문,URL,topic,keyword,summary_text,위험도,city
49,19388,2.100311e+06,2023-08-29,尹 카눈 피해 심각 지자체 3곳 특별재난지역 추가 선포,윤석열 대통령이 제6호 태풍 카눈으로 인해 대규모 피해가 발생한 강원 고성군 경북 ...,http://www.sedaily.com/NewsView/29TLG4GC12,3,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",제6호 태풍 카눈으로 인해 대규모 피해가 발생한 강원 고성군 경북 경주시 산내면 경...,12.492217,경상북도
60,19480,2.100501e+06,2023-08-29,'살인 예고글' 전국서 479건 23명 구속,신림동 흉기난동 이후 229건 검거 \n \n 대구국제공항에 대한 '폭탄테러'와 공...,http://www.fnnews.com/news/202308290954412819,3,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",대구국제공항에 대한 '폭탄테러'와 공항 이용객 불특정 다수를 상대로 '흉기난동'이 ...,1.554176,전라북도
54,19282,2.100851e+06,2023-08-29,10호 태풍 '담레이' 일본 삿포로 해상에서 소멸,제10호 태풍 '담레이' 경로. 담레이는 29일 오후 3시에 소멸했다. [자료=기상...,https://www.ajunews.com/view/20230829171154152,1,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",29일 낮 낮 일본 삿포로 인근 해상에서 소멸된 제10호 태풍 '담레이'가 29일 ...,12.258479,대전광역시
44,19317,7.100501e+06,2023-08-29,"한난, '2023년 재난대응 안전한국훈련' 시행",한국지역난방공사는 29일 경기 파주시 지사에서 행정안전부가 주관하는 '2023년 제...,http://www.etnews.com/20230829000284,-1,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...","한국지역난방공사는 29일 경기 파주시 지사에서 중앙부처와 지방자치단체, 공공기관, ...",8.469362,세종특별자치시
9,18922,1.100701e+06,2023-08-30,日 “간토대지진 조선인 학살 사실 파악 기록 없어”,"일본 정부가 올해로 100주년을 맞은 간토대지진 당시 군대와 경찰, 민간인으로 이뤄...",http://www.segye.com/content/html/2023/08/30/2...,2,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...","일본 정부가 올해로 100주년을 맞은 간토대지진 당시 군대와 경찰, 민간인으로 이뤄...",2.554163,대구광역시
...,...,...,...,...,...,...,...,...,...,...,...
19,786,2.100501e+06,2023-11-03,전세계 울린 “다섯 아이 구해 나온 '가자지구 아버지'...가짜였다 '충격',지난달 28일 프랑스 주재 중국 대사관이 공유한 사진. 인공지능(AI)을 동원한 가...,http://www.fnnews.com/news/202311030642049849,1,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...","인공지능(AII)을 동원한 가짜 사진으로 판명났지만, 2일 현재 내리지 않고 있으며...",2.066899,충청남도
77,845,1.400701e+06,2023-11-03,[국비 확보전 본격화] 세종의사당 대통령 제2집무실 추진 행정수도 인프라 조성 '핵심',[충청투데이 강대묵 기자] 세종시의 내년도 정부예산안의 핵심 포인트는 ‘행정수도 인...,https://www.cctoday.co.kr/news/articleView.htm...,3,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",세종시의 내년도 정부예산안의 핵심 포인트는 ‘행정수도 인프라 조성’이며 국회 세종의...,6.152156,제주특별자치도
68,573,8.100401e+06,2023-11-03,"""휴전은 없다!"" 가자시티 포위...대량살상에 확전우려까지",[앵커]\n이스라엘이 가자시티를 포위하면서 하마스와의 전쟁이 본격적인 시가전 국면에...,https://www.ytn.co.kr/_ln/0104_202311032220204506,1,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",이스라엘이 가자시티를 포위하면서 하마스와의 전쟁이 본격적인 시가전 국면에 돌입하였으...,12.807173,강원도
2,486,1.100101e+06,2023-11-04,하마스 “이스라엘의 학교 겨냥 공격으로 20명 사망”,팔레스타인 가자지구의 학교를 겨냥한 이스라엘군의 공습으로 20명이 사망했다고 팔레스...,https://www.khan.co.kr/world/world-general/art...,1,"[홍수위로부터, 홍수, 환경분쟁조정을, 피해가, 계획, 지역까지, 지원, 지역, 지...",팔레스타인 가자지구의 학교를 겨냥한 이스라엘군의 공습으로 20명이 사망했다고 팔레스...,9.579531,강원도


In [170]:
trans_article = tokenizer.batch_decode(generated_article, skip_special_tokens=True)
trans_title = tokenizer.batch_decode(generated_title, skip_special_tokens=True)
#trans_topic = tokenizer.batch_decode(generated_topic, skip_special_tokens=True)

In [173]:
trans_article

["[Manya Today, PADO Editor-in-Chief] [Editor-in-Chief] The Palestinian armed group Hamas has launched a massive attack on Israel, launching thousands of rockets, and then the ground forces have invaded Israeli territory and brutally murdered and abducted civilians, perhaps killing civilians, and exposing the scenes of these murders has provoked Israel's retaliatory response to the war that was waged between Israel and the Arabs."]

## 위 과정을 함수로 작성

In [277]:
def translate(df, city, topic, start_date, end_date, output_language) :
    return_df = df.loc[df["city"] == input_city]
    return_df = return_df[return_df["topic"] == input_topic]
    return_df = return_df.loc[return_df.일자.between(input_date_start, input_date_end)]
    return_df.reset_index(drop = True, inplace = True)
    if len(return_df) == 0 : print("No Result")
    
    else : 
        trans_date = []
        trans_topic = []
        trans_title = []
        trans_keyword = []
        trans_article = []
        trans_summary = []
        for i in range(len(return_df)) :
            summary_kr = return_df["summary_text"][i]
            article_kr = return_df["본문"][i]
            title_kr = return_df["제목"][i]
            keyword_kr = return_df["keyword"][i]
            topic_kr = return_df["topic"][i]
            date = return_df["일자"][i]
            
            
            language = lang_dict[output_language] ## output 언어 설정
            tokenizer.src_lang = "ko_KR" ## input 한국어로 고정
            
            encoded_summary_kr = tokenizer(summary_kr, return_tensors = "pt")
            encoded_article_kr = tokenizer(article_kr, return_tensors= "pt")
            encoded_title_kr = tokenizer(title_kr, return_tensors= "pt")
            encoded_keyword_kr = tokenizer(keyword_kr, return_tensors= "pt")
            #encoded_topic_kr = tokenizer(topic_kr, return_tensors="pt")
            
            generated_article = model.generate(
                            **encoded_article_kr,
                            forced_bos_token_id = tokenizer.lang_code_to_id[language])

            generated_title = model.generate(
                            **encoded_title_kr,
                            forced_bos_token_id = tokenizer.lang_code_to_id[language])
            
            generated_keyword = model.generate(
                            **encoded_keyword_kr,
                            forced_bos_token_id = tokenizer.lang_code_to_id[language])
            
            generated_summary = model.generate(
                            **encoded_summary_kr,
                            forced_bos_token_id = tokenizer.lang_code_to_id[language])
            
            #generated_topic = model.generate(
     #                       **encoded_topic_kr,
      #                      forced_bos_token_id = tokenizer.lang_code_to_id[language])
            
            trans_date.append(date)
            trans_article.append(tokenizer.batch_decode(generated_article, skip_special_tokens=True))
            trans_title.append(tokenizer.batch_decode(generated_title, skip_special_tokens=True))
            trans_keyword.append(tokenizer.batch_decode(generated_keyword, skip_special_tokens=True))
            trans_summary.append(tokenizer.batch_decode(generated_summary, skip_special_tokens=True))
            
            #trans_topic.append(tokenizer.batch_decode(generated_topic, skip_special_tokens=True))
        trans_df = pd.DataFrame({"date" : trans_date,
                                 "title" : trans_title,
                                 "topic" : topic_kr,
                                 "keyword" : trans_keyword, 
                                 "article" : trans_article,
                                 "summary_text" : trans_summary})
        return trans_df

In [273]:
city = "강원도"
topic = 1
start_date = "2023-10-14"
end_date = "2023-11-01"
output_language = "English"

In [278]:
result = translate(df, city, topic, start_date, end_date, output_language)

In [279]:
result

,date,title,topic,keyword,article,summary_text
0,2023-10-14,[A former U.S. ambassador to Israel analyzing ...,1,"[Symptoms are parasitic, nectar nectar nectar,...","[[Manya Today, PADO Editor-in-Chief] [Editor-i...",[The Palestinian armed group Hamas had launche...
1,2023-10-16,[Iranian Revolutionary Guards Move Closer to I...,1,[The Israeli-Palestinian conflict in Gaza has ...,[In the wake of reports that Syrian troops fro...,[With the possibility of an Israeli-Hamas war ...
2,2023-10-26,"[""You must remove the enemy's heart and liver,...",1,"[Buildings, urban buildings, African villages,...",[In the midst of the Palestinian armed Hamas a...,[In the midst of the Palestinian armed Hamas a...


`-` 아 마지막 이렇게 결과 반환하는 거 너무 오래걸린다....

In [280]:
print(result["title"][0][0])

A former U.S. ambassador to Israel analyzing the Israeli-Hamas War [PADO].


In [281]:
print(result["keyword"][0][0])

Symptoms are parasitic, nectar nectar nectar, 487 weeks of bugs, 18 days in the trees, 21 days in August improvements, most of which are feces and methasoccus.


In [282]:
print(result["summary_text"][0][0])

The Palestinian armed group Hamas had launched a massive attack on Israel, fired thousands of missiles, invaded Israeli territory, and brutally murdered and abducted civilians, and even killed and maimed civilians, exposing the scene of the killing, provoking an Israeli retaliatory campaign called the Israeli-Arab War, the Israeli-Arab War.


In [266]:
print(result["article"][0][0])

[Manya Today, PADO Editor-in-Chief] [Editor-in-Chief] The Palestinian armed group Hamas has launched a massive attack on Israel, launching thousands of rockets, and then the ground forces have invaded Israeli territory and brutally murdered and abducted civilians, perhaps killing civilians, and exposing the scenes of these murders has provoked Israel's retaliatory response to the war that was waged between Israel and the Arabs.
